In [3]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data=pd.read_csv("../artifacts/data/sp500.csv",skipfooter=40000)
data.head()

C:\Users\manth\AppData\Local\Temp\ipykernel_13056\2540032479.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data=pd.read_csv("../artifacts/data/sp500.csv",skipfooter=40000)


,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    10000 non-null  object 
 1   open    10000 non-null  float64
 2   high    10000 non-null  float64
 3   low     10000 non-null  float64
 4   close   10000 non-null  float64
 5   volume  10000 non-null  int64  
 6   Name    10000 non-null  object 
dtypes: float64(4), int64(1), object(2)
memory usage: 547.0+ KB


In [6]:
data.describe()


,open,high,low,close,volume
count,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04
mean,81.409088,82.164968,80.663020,81.443147,1.092780e+07
std,37.828416,38.119626,37.523002,37.836402,2.072435e+07
min,13.140000,13.420000,12.700000,13.020000,1.307120e+05
25%,48.740000,49.183750,48.280000,48.677500,1.897144e+06
50%,76.075000,76.800000,75.485050,76.145000,4.015270e+06
75%,105.742500,106.825000,104.861250,105.862500,8.631211e+06
max,201.240000,201.240000,198.160000,200.380000,2.668336e+08


In [7]:
data.isna().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
Name      0
dtype: int64

In [8]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')
data['open']=imputer.fit_transform(data[['open']])
data['high']=imputer.fit_transform(data[['high']])
data['low']=imputer.fit_transform(data[['low']])

data.isna().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
Name      0
dtype: int64

In [9]:
data.duplicated().sum()

np.int64(0)

In [10]:
from sklearn.preprocessing import  LabelEncoder
encoder=LabelEncoder()
data['Name']=encoder.fit_transform(data['Name'])


In [11]:
data['date'] = pd.to_datetime(data['date'])

# Day, month, year
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

# Day of the week (0 = Monday, 6 = Sunday)
data['day_of_week'] = data['date'].dt.dayofweek

# Week of the year
data['week_of_year'] = data['date'].dt.isocalendar().week

# Quarter of the year
data['quarter'] = data['date'].dt.quarter

# Is month start/end
data['is_month_start'] = data['date'].dt.is_month_start.astype(int)
data['is_month_end'] = data['date'].dt.is_month_end.astype(int)

# Is quarter start/end
data['is_quarter_start'] = data['date'].dt.is_quarter_start.astype(int)
data['is_quarter_end'] = data['date'].dt.is_quarter_end.astype(int)



In [12]:
cols = ['open', 'high', 'low', 'close']
windows = [5, 10, 20]

for col in cols:
    for w in windows:
        data[f'{col}_EMA_{w}'] = data[col].ewm(span=w, adjust=False).mean()


In [13]:
data.head()

,date,open,high,low,close,volume,Name,year,month,day,...,open_EMA_20,high_EMA_5,high_EMA_10,high_EMA_20,low_EMA_5,low_EMA_10,low_EMA_20,close_EMA_5,close_EMA_10,close_EMA_20
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,0,2013,2,8,...,15.070000,15.120000,15.120000,15.120000,14.630000,14.630000,14.630000,14.750000,14.750000,14.750000
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,0,2013,2,11,...,15.052857,15.083333,15.100000,15.109524,14.506667,14.562727,14.594762,14.653333,14.697273,14.722381
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,0,2013,2,12,...,14.995442,14.892222,14.992727,15.052426,14.371111,14.478595,14.547642,14.525556,14.619587,14.679297
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,0,2013,2,13,...,14.929210,14.908148,14.983140,15.041719,14.330741,14.437032,14.519295,14.570370,14.626935,14.677459
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,0,2013,2,14,...,14.930237,14.925432,14.978933,15.033936,13.940494,14.204845,14.389838,14.376914,14.511128,14.611987


In [14]:
X=data.drop(columns=['date','close']).values
Y=data['close'].values

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, shuffle=False)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV

# Regression models
from sklearn.linear_model import LinearRegression ,Ridge , Lasso, ElasticNet    
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor

# XGBoost, LightGBM, CatBoost
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [18]:
models={
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "GradientBoostingRegressor": GradientBoostingRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),
    "BaggingRegressor": BaggingRegressor(),
    "XGBRegressor": XGBRegressor(objective='reg:squarederror'),
    "LGBMRegressor": LGBMRegressor(),
    "CatBoostRegressor": CatBoostRegressor(verbose=0),
    "SVR": SVR(),
}

param_grids = {

    "Ridge": {
        "alpha": [0.01, 0.1, 1.0, 10.0, 100.0],
        "solver": ["auto", "svd", "cholesky", "lsqr", "sag", "saga"]
    },

    "Lasso": {
        "alpha": [0.001, 0.01, 0.1, 1.0, 10.0],
        "selection": ["cyclic", "random"]
    },

    "ElasticNet": {
        "alpha": [0.001, 0.01, 0.1, 1.0, 10.0],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
    },

    "KNeighborsRegressor": {
        "n_neighbors": [3, 5, 7, 9, 15],
        "weights": ["uniform", "distance"],
        "p": [1, 2]  # 1=Manhattan, 2=Euclidean
    },

    "DecisionTreeRegressor": {
        "criterion": ["squared_error", "friedman_mse", "absolute_error"],
        "splitter": ["best", "random"],
        "max_depth": [None, 5, 10, 20, 50],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 5, 10]
    },

    "RandomForestRegressor": {
        "n_estimators": [100, 200, 500],
        "criterion": ["squared_error", "absolute_error"],
        "max_depth": [None, 10, 20, 50],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": ["auto", "sqrt", "log2"]
    },

    "GradientBoostingRegressor": {
        "n_estimators": [100, 200, 500],
        "learning_rate": [0.01, 0.05, 0.1, 0.2],
        "subsample": [0.6, 0.8, 1.0],
        "max_depth": [3, 5, 10],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    },

    "AdaBoostRegressor": {
        "n_estimators": [50, 100, 200, 500],
        "learning_rate": [0.01, 0.1, 0.5, 1.0],
        "loss": ["linear", "square", "exponential"]
    },

    "BaggingRegressor": {
        "n_estimators": [10, 50, 100, 200],
        "max_samples": [0.5, 0.7, 1.0],
        "max_features": [0.5, 0.7, 1.0],
        "bootstrap": [True, False]
    },

    "XGBRegressor": {
        "n_estimators": [100, 200, 500],
        "learning_rate": [0.01, 0.05, 0.1, 0.2],
        "max_depth": [3, 5, 7, 10],
        "subsample": [0.6, 0.8, 1.0],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "gamma": [0, 0.1, 0.2, 0.3],
        "reg_alpha": [0, 0.01, 0.1, 1],
        "reg_lambda": [0.1, 1.0, 10.0]
    },

    "LGBMRegressor": {
        "n_estimators": [100, 200, 500],
        "learning_rate": [0.01, 0.05, 0.1],
        "num_leaves": [31, 50, 100],
        "max_depth": [-1, 5, 10, 20],
        "subsample": [0.6, 0.8, 1.0],
        "colsample_bytree": [0.6, 0.8, 1.0],
        "reg_alpha": [0, 0.01, 0.1],
        "reg_lambda": [0, 0.1, 1.0]
    },
    
    "CatBoostRegressor": {
        "iterations": [200, 500, 1000],
        "learning_rate": [0.01, 0.05, 0.1],
        "depth": [4, 6, 8, 10],
        "l2_leaf_reg": [1, 3, 5, 7],
        "bagging_temperature": [0, 0.5, 1.0]
    },
    

    "SVR": {
        "kernel": ["linear", "poly", "rbf", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": ["scale", "auto"],
        "degree": [2, 3, 4]
    }
}




In [19]:
result={}
def model_training(models,param_grids,x_train,y_train,x_test,y_test):
    for name, model in models.items():
        print(f"Training {name}...")
        param_grid = param_grids.get(name, {})
        if param_grid:
            grid_search = RandomizedSearchCV(model, param_grid, cv=2, scoring='neg_mean_squared_error', n_jobs=-1)
            grid_search.fit(x_train, y_train)
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            print(f"Best parameters for {name}: {best_params}")
        else:
            model.fit(x_train, y_train)
            best_model = model
            best_params = "Default parameters used"
        
        y_pred = best_model.predict(x_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        result[name] = {
            'model': best_model,
            'best_params': best_params,
            'mse': mse,
            'mae': mae,
            'r2': r2
        }
        
        print(f"{name} - MSE: {mse}, MAE: {mae}, R2: {r2}\n")
        
    return result

In [20]:
model_training(models,param_grids,x_train,y_train,x_test,y_test)

Training LinearRegression...
LinearRegression - MSE: 0.04927540612668112, MAE: 0.16350980697370893, R2: 0.9999424387088994

Training Ridge...
Best parameters for Ridge: {'solver': 'svd', 'alpha': 0.01}
Ridge - MSE: 0.07957917376549627, MAE: 0.20347482330981786, R2: 0.9999070392240932

Training Lasso...


d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.987e+03, tolerance: 1.203e+03
  model = cd_fast.enet_coordinate_descent(


Best parameters for Lasso: {'selection': 'random', 'alpha': 0.01}
Lasso - MSE: 0.7000205384841668, MAE: 0.5553857429382325, R2: 0.9991822678053938

Training ElasticNet...


d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.491e+03, tolerance: 1.203e+03
  model = cd_fast.enet_coordinate_descent(


Best parameters for ElasticNet: {'l1_ratio': 0.7, 'alpha': 0.001}
ElasticNet - MSE: 0.5557524610375959, MAE: 0.5406056345057181, R2: 0.9993507952200857

Training KNeighborsRegressor...
Best parameters for KNeighborsRegressor: {'weights': 'uniform', 'p': 1, 'n_neighbors': 3}
KNeighborsRegressor - MSE: 9.200761804671668, MAE: 2.16785605, R2: 0.9892520880046244

Training DecisionTreeRegressor...
Best parameters for DecisionTreeRegressor: {'splitter': 'random', 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10, 'criterion': 'squared_error'}
DecisionTreeRegressor - MSE: 2.8944582545070805, MAE: 0.6881332458332997, R2: 0.9966188253479256

Training RandomForestRegressor...


d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
8 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "d:\Pr

Best parameters for RandomForestRegressor: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'squared_error'}
RandomForestRegressor - MSE: 1.1211083386903364, MAE: 0.5679363972834883, R2: 0.9986903721651171

Training GradientBoostingRegressor...
Best parameters for GradientBoostingRegressor: {'subsample': 0.8, 'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10, 'learning_rate': 0.05}
GradientBoostingRegressor - MSE: 0.27917943997020006, MAE: 0.3930500711894422, R2: 0.9996738752599601

Training AdaBoostRegressor...
Best parameters for AdaBoostRegressor: {'n_estimators': 100, 'loss': 'square', 'learning_rate': 1.0}
AdaBoostRegressor - MSE: 1.884724423987958, MAE: 1.1021319998321515, R2: 0.9977983505415527

Training BaggingRegressor...
Best parameters for BaggingRegressor: {'n_estimators': 10, 'max_samples': 0.7, 'max_features': 0.7, 'bootstrap': True}
BaggingRegressor - MSE: 0.371939

d:\Projects\Portfolio Project\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Best parameters for CatBoostRegressor: {'learning_rate': 0.01, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 4, 'bagging_temperature': 1.0}
CatBoostRegressor - MSE: 2.101742193373927, MAE: 0.9946883986581477, R2: 0.9975448402413938

Training SVR...
Best parameters for SVR: {'kernel': 'linear', 'gamma': 'auto', 'degree': 4, 'C': 10}
SVR - MSE: 0.16379049225141026, MAE: 0.22985696128343172, R2: 0.9998086673871392



{'LinearRegression': {'model': LinearRegression(),
  'best_params': 'Default parameters used',
  'mse': 0.04927540612668112,
  'mae': 0.16350980697370893,
  'r2': 0.9999424387088994},
 'Ridge': {'model': Ridge(alpha=0.01, solver='svd'),
  'best_params': {'solver': 'svd', 'alpha': 0.01},
  'mse': 0.07957917376549627,
  'mae': 0.20347482330981786,
  'r2': 0.9999070392240932},
 'Lasso': {'model': Lasso(alpha=0.01, selection='random'),
  'best_params': {'selection': 'random', 'alpha': 0.01},
  'mse': 0.7000205384841668,
  'mae': 0.5553857429382325,
  'r2': 0.9991822678053938},
 'ElasticNet': {'model': ElasticNet(alpha=0.001, l1_ratio=0.7),
  'best_params': {'l1_ratio': 0.7, 'alpha': 0.001},
  'mse': 0.5557524610375959,
  'mae': 0.5406056345057181,
  'r2': 0.9993507952200857},
 'KNeighborsRegressor': {'model': KNeighborsRegressor(n_neighbors=3, p=1),
  'best_params': {'weights': 'uniform', 'p': 1, 'n_neighbors': 3},
  'mse': 9.200761804671668,
  'mae': 2.16785605,
  'r2': 0.9892520880046244

In [21]:
best_parameters={}
for i in result.keys():
    best_parameters[i]=result[i]['best_params']


    

In [22]:
best_parameters

{'LinearRegression': 'Default parameters used',
 'Ridge': {'solver': 'svd', 'alpha': 0.01},
 'Lasso': {'selection': 'random', 'alpha': 0.01},
 'ElasticNet': {'l1_ratio': 0.7, 'alpha': 0.001},
 'KNeighborsRegressor': {'weights': 'uniform', 'p': 1, 'n_neighbors': 3},
 'DecisionTreeRegressor': {'splitter': 'random',
  'min_samples_split': 5,
  'min_samples_leaf': 2,
  'max_depth': 10,
  'criterion': 'squared_error'},
 'RandomForestRegressor': {'n_estimators': 500,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'max_features': 'log2',
  'max_depth': 10,
  'criterion': 'squared_error'},
 'GradientBoostingRegressor': {'subsample': 0.8,
  'n_estimators': 500,
  'min_samples_split': 5,
  'min_samples_leaf': 2,
  'max_depth': 10,
  'learning_rate': 0.05},
 'AdaBoostRegressor': {'n_estimators': 100,
  'loss': 'square',
  'learning_rate': 1.0},
 'BaggingRegressor': {'n_estimators': 10,
  'max_samples': 0.7,
  'max_features': 0.7,
  'bootstrap': True},
 'XGBRegressor': {'subsample': 0.8,
  '

In [23]:
import json

with open('../artifacts/best_params/best_params.json', 'w') as f:
    json.dump(best_parameters, f)
